In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/seungheondoh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [85]:
client = MongoClient()
db = client.aboutdepart
database = list(db.Docdb.find())
testdf = pd.DataFrame(database)
testdf.head()

,Abstract,Author Keywords,Name,ResearchInterest,Title,_id,department,docs,text
0,[Entrepreneurs' decisions to exploit opportuni...,"[nan, Behavioral decision-making; Business gro...","Choi, Young Rok(최영록)","Entrepreneurship, Innovation Strategy, Family ...",[Entrepreneurs' decisions to exploit opportuni...,5bf7bdc420c66c03397b5abd,Graduate School of Technology and Innovation M...,[[Entrepreneurs' decisions to exploit opportun...,Entrepreneurs' decisions to exploit opportuni...
1,[Organizations use project teams to lower sear...,[expertise awareness; expertise use; knowledge...,"Hong, Woonki(홍운기)","Expertise utilization, Transactive menory syst...",[Explaining dyadic expertise use in knowledge ...,5bf7bdc520c66c03397b5abe,School of Business Administration(경영학부),[[Explaining dyadic expertise use in knowledge...,Explaining dyadic expertise use in knowledge ...
2,[This paper investigates information leakage f...,"[nan, nan, Business group; Chaebol; Control-ca...","Jung, Kooyul(정구열)","Accounting Information and capital market, Ana...",[Trading behavior prior to public release of a...,5bf7bdc520c66c03397b5abf,Graduate School of Technology and Innovation M...,[[Trading behavior prior to public release of ...,Trading behavior prior to public release of a...
3,[Studies on the effects of corporate social re...,[annual reports; communication; corporate soci...,"Kim, MinChung(김민중)",Marketing strategies and their impact on finan...,[CSR and Shareholder Value in the Restaurant I...,5bf7bdc520c66c03397b5ac0,School of Business Administration(경영학부),[[CSR and Shareholder Value in the Restaurant ...,CSR and Shareholder Value in the Restaurant I...
4,[Brand communities and corporate social respon...,[Brand community; Consumer communities; Corpor...,"Kim, Molan(김모란)","Online Database Marketing, Social Media Market...","[Consumer Communities Do Well, But Will They D...",5bf7bdc520c66c03397b5ac1,School of Business Administration(경영학부),"[[Consumer Communities Do Well, But Will They ...","Consumer Communities Do Well, But Will They D..."


In [5]:
testdf.to_csv('prof2vec.csv')

In [86]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords


def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    

    docs = docs.lower()  # Convert to lowercase.
    docs = tokenizer.tokenize(docs)  # Split into words.
    docs = [w for w in docs if not w in stopwords.words('english')]
    
    # Remove numbers, but not words that contain numbers.
    docs = [token for token in docs if not token.isdigit()]
    
    # Remove words that are only one character.
    docs = [token for token in docs if len(token) > 3]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [lemmatizer.lemmatize(token) for token in docs]
  
    return docs

In [87]:
for idx, i in enumerate(testdf['text']):
    add = docs_preprocessor(i)
    testdf.iloc[idx]['text'] = add

In [88]:
for idx, i in enumerate(testdf['Name']):
    testdf.iloc[idx]['Name'] = i.split("(")[0]

In [89]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [90]:
zipdata = zip(testdf['Name'],testdf['text'])

In [91]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in zipdata]

In [92]:
tagged_data[3]

TaggedDocument(words=['shareholder', 'value', 'restaurant', 'industry', 'role', 'communication', 'annual', 'report', 'study', 'effect', 'corporate', 'social', 'responsibility', 'shareholder', 'value', 'identified', 'communication', 'marketing', 'channel', 'mechanism', 'necessary', 'translate', 'shareholder', 'value', 'study', 'annual', 'report', 'considered', 'primary', 'information', 'source', 'used', 'firm', 'financial', 'stakeholder', 'investor', 'analyst', 'valuation', 'firm', 'specifically', 'study', 'examined', 'whether', 'extent', 'restaurant', 'firm', 'communicate', 'annual', 'report', 'influence', 'effect', 'shareholder', 'value', 'return', 'tobin', 'risk', 'equity', 'holder', 'risk', 'disaggregated', 'distinct', 'type', 'primary', 'nonprimary', 'stakeholder', 'result', 'corroborate', 'communicating', 'nonprimary', 'stakeholder', 'annual', 'report', 'help', 'nonprimary', 'stakeholder', 'increase', 'shareholder', 'value', 'reducing', 'equity', 'holder', 'risk', 'whereas', 'comm

In [93]:
import re 
import gensim
from gensim.models import doc2vec
from collections import namedtuple
from random import shuffle

In [94]:
import warnings
warnings.filterwarnings("ignore")

In [95]:
model = gensim.models.Doc2Vec(vector_size=100, window=10, min_count=1, workers=8, alpha=0.025, min_alpha=0.015, 
                              epochs=20)

#sample=1e-4, negative=5,

#shuffling is better (ot needed at each trianing epoch
shuffle(tagged_data)
#Build vocabulary from a sequence of sentences 
model.build_vocab(tagged_data)
#Update the model’s neural weights from a sequence of sentences
model.train(tagged_data, epochs=model.epochs, total_examples=model.corpus_count)

2018-12-05 10:30:09,764 - doc2vec - INFO - collecting all words and their counts
2018-12-05 10:30:09,767 - doc2vec - INFO - PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-12-05 10:30:10,323 - doc2vec - INFO - collected 39597 word types and 258 unique tags from a corpus of 258 examples and 1570282 words
2018-12-05 10:30:10,324 - word2vec - INFO - Loading a fresh vocabulary
2018-12-05 10:30:11,495 - word2vec - INFO - effective_min_count=1 retains 39597 unique words (100% of original 39597, drops 0)
2018-12-05 10:30:11,495 - word2vec - INFO - effective_min_count=1 leaves 1570282 word corpus (100% of original 1570282, drops 0)
2018-12-05 10:30:11,627 - word2vec - INFO - deleting the raw counts dictionary of 39597 items
2018-12-05 10:30:11,628 - word2vec - INFO - sample=0.001 downsamples 21 most-common words
2018-12-05 10:30:11,629 - word2vec - INFO - downsampling leaves estimated 1546542 word corpus (98.5% of prior 1570282)
2018-12-05 10:30:11,850 - base_any2ve

2018-12-05 10:30:25,136 - base_any2vec - INFO - worker thread finished; awaiting finish of 1 more threads
2018-12-05 10:30:25,148 - base_any2vec - INFO - worker thread finished; awaiting finish of 0 more threads
2018-12-05 10:30:25,152 - base_any2vec - INFO - EPOCH - 6 : training on 1570282 raw words (1266148 effective words) took 2.2s, 564554 effective words/s
2018-12-05 10:30:26,168 - base_any2vec - INFO - EPOCH 7 - PROGRESS: at 58.91% examples, 710602 words/s, in_qsize 15, out_qsize 0
2018-12-05 10:30:26,817 - base_any2vec - INFO - worker thread finished; awaiting finish of 7 more threads
2018-12-05 10:30:26,831 - base_any2vec - INFO - worker thread finished; awaiting finish of 6 more threads
2018-12-05 10:30:26,832 - base_any2vec - INFO - worker thread finished; awaiting finish of 5 more threads
2018-12-05 10:30:26,843 - base_any2vec - INFO - worker thread finished; awaiting finish of 4 more threads
2018-12-05 10:30:26,845 - base_any2vec - INFO - worker thread finished; awaiting fi

2018-12-05 10:30:39,387 - base_any2vec - INFO - worker thread finished; awaiting finish of 1 more threads
2018-12-05 10:30:39,389 - base_any2vec - INFO - worker thread finished; awaiting finish of 0 more threads
2018-12-05 10:30:39,390 - base_any2vec - INFO - EPOCH - 13 : training on 1570282 raw words (1266099 effective words) took 2.3s, 554054 effective words/s
2018-12-05 10:30:40,406 - base_any2vec - INFO - EPOCH 14 - PROGRESS: at 38.37% examples, 486519 words/s, in_qsize 15, out_qsize 0
2018-12-05 10:30:41,431 - base_any2vec - INFO - EPOCH 14 - PROGRESS: at 84.50% examples, 534279 words/s, in_qsize 13, out_qsize 2
2018-12-05 10:30:41,680 - base_any2vec - INFO - worker thread finished; awaiting finish of 7 more threads
2018-12-05 10:30:41,682 - base_any2vec - INFO - worker thread finished; awaiting finish of 6 more threads
2018-12-05 10:30:41,683 - base_any2vec - INFO - worker thread finished; awaiting finish of 5 more threads
2018-12-05 10:30:41,692 - base_any2vec - INFO - worker th

2018-12-05 10:30:53,453 - base_any2vec - INFO - worker thread finished; awaiting finish of 0 more threads
2018-12-05 10:30:53,454 - base_any2vec - INFO - EPOCH - 20 : training on 1570282 raw words (1266433 effective words) took 1.7s, 742158 effective words/s
2018-12-05 10:30:53,455 - base_any2vec - INFO - training on a 31405640 raw words (25324816 effective words) took 40.7s, 621784 effective words/s


In [15]:
model.save("finalmodel")

In [96]:
model.docvecs.doctags

{'Baek, Woongki': Doctag(offset=0, word_count=3611, doc_count=1),
 'Son, Jae Sung': Doctag(offset=1, word_count=3129, doc_count=1),
 'Kim, Jooha': Doctag(offset=2, word_count=933, doc_count=1),
 'Kim, Guntae': Doctag(offset=3, word_count=10826, doc_count=1),
 'Kim, Chae Un': Doctag(offset=4, word_count=321, doc_count=1),
 'Park, Hyesung': Doctag(offset=5, word_count=2059, doc_count=1),
 'Chun, Se Young': Doctag(offset=6, word_count=4987, doc_count=1),
 'Lim, Sunghoon': Doctag(offset=7, word_count=867, doc_count=1),
 'Song, Hyun-Kon': Doctag(offset=8, word_count=10331, doc_count=1),
 'Jung, Jee-Hoon': Doctag(offset=9, word_count=7931, doc_count=1),
 'Chung, Moses': Doctag(offset=10, word_count=3529, doc_count=1),
 'Myung, Kyungjae': Doctag(offset=11, word_count=9148, doc_count=1),
 'Lee, Changwook': Doctag(offset=12, word_count=2439, doc_count=1),
 'Kim, Jaai': Doctag(offset=13, word_count=6645, doc_count=1),
 'Kwon, Daeil': Doctag(offset=14, word_count=4439, doc_count=1),
 'Kim, Byungm

In [97]:
model.docvecs.most_similar('Kim, Chajoong')

2018-12-05 10:34:14,510 - keyedvectors - INFO - precomputing L2-norms of doc weight vectors


[('Self, James', 0.7371705770492554),
 ('Baek, Joon Sang', 0.7270042300224304),
 ('Kim, KwanMyung', 0.6994534730911255),
 ('Lee, Jacob C.', 0.6763160228729248),
 ('Lim, Sunghoon', 0.6498798131942749),
 ('Kyung, Gyouhyung', 0.648121178150177),
 ('Zhang, Lu', 0.6405069231987),
 ('Kim, Yeolib', 0.6356873512268066),
 ('Lim, Chiehyeon', 0.6298955678939819),
 ('Kim, Molan', 0.6261118054389954)]

In [98]:
model.save_word2vec_format('doc_tensor.w2v', doctag_vec=True, word_vec=False, binary=True)  

2018-12-05 10:34:24,572 - keyedvectors - INFO - storing 258x100 projection weights into doc_tensor.w2v


In [18]:
profname = list(model.docvecs.doctags.keys())

In [106]:
with open('outfiletsv.tsv', 'w') as file_vector:
    with open('outfiletsvmeta.tsv', 'w') as file_metadata:
        for word in model.docvecs.index2entity:
            file_metadata.write(word + '\n')
            vector_row = '\t'.join(str(x) for x in model[word])
            file_vector.write(vector_row + '\n')

In [48]:
with open('prof2vec.tsv','w') as file_vector:
    for word in model.docvecs.index2entity:
        vector_row = '\t'.join(str(x) for x in model[word])
        file_vector.write(vector_row + '\n')

In [39]:
%run /Users/seungheondoh/anaconda3/envs/python3.6/lib/python3.6/site-packages/gensim/scripts/word2vec2tensor.py -i doc_tensor.w2v -o prof2vec

2018-12-05 09:45:52,754 - word2vec2tensor - INFO - running /Users/seungheondoh/anaconda3/envs/python3.6/lib/python3.6/site-packages/gensim/scripts/word2vec2tensor.py -i doc_tensor.w2v -o prof2vec
2018-12-05 09:45:52,756 - utils_any2vec - INFO - loading projection weights from doc_tensor.w2v


ValueError: invalid vector on line 0 (is this really the text format?)

In [33]:
with open('prof2vec_metadata.tsv','w') as w:
    w.write('Titles\tdepartment\n')
    for i,j in zip(testdf.Name, testdf.department):
        w.write("%s\t%s\n" % (i,j))

In [220]:
final = []
for i in profname:
    count = 0
    mostsimilar = model.docvecs.most_similar(i)
    for a,b in mostsimilar:
        if a.split('/')[1] == i.split('/')[1]:
            count += 1
        else:
            pass
    final.append(count)

/Users/seungheondoh/anaconda3/envs/python3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [109]:
def parametertunning(v):
    model = gensim.models.Doc2Vec(vector_size=v, window=10, min_count=1, workers=8, alpha=0.025, min_alpha=0.015, 
                              epochs=20)
    profname = list(model.docvecs.doctags.keys())
    final = []
    for j in profname:
        count = 0
        mostsimilar = model.docvecs.most_similar(j)
        for a,b in mostsimilar:
            if a.split('/')[1] == i.split('/')[1]:
                count += 1
            else:
                pass
        final.append(count)
        average = sum(final)/len(final)
        return average